In [7]:
from main import *
USE_CACHE = True
TRAIT_NAMES = ['NRG', 'AGG', 'SPK', 'BRN', 'EYS', 'EYC']
SLOT_POSITION_NAMES = ['Body', 'Face', 'Eyes', 'Head', 'Hand L', 'Hand R', 'Pet', 'BG']
RARITY_SCORE_MODIFIERS = { 1: 'common', 2: 'uncommon', 5: 'rare', 10: 'legendary', 20: 'mythical', 50: 'godlike'  }
UPDATE_TIME_HASH = hex(hash(UPDATE_TIME))

In [8]:
wearables_query = get_core_matic_query(
    { 'itemTypes': {
        'params': { 'where': { 'category': ITEM_TYPE_CATEGORY_WEARABLE, 'id_not': UPDATE_TIME_HASH, 'id_not': 0, 'canBeTransferred': True } },
        'fields': ["id", "name", "traitModifiers", "slotPositions", "maxQuantity", "rarityScoreModifier"] }}
)

wearables_result = wearables_query.execute(USE_CACHE)
wearables_df = get_subgraph_result_df(wearables_result)
wearables_df.set_index(wearables_df.index.astype(np.int64), inplace=True)
wearables_df.sort_index(inplace=True)
item_types_int_fields = ['maxQuantity', 'rarityScoreModifier']
wearables_df[item_types_int_fields] = wearables_df[item_types_int_fields].astype(np.int64)

# trait modifiers
append = lambda s2: lambda s1: str(s1) + str(s2)
traitEffectColumns = list(map(append(' Effect'), TRAIT_NAMES))
getTraitEffect = lambda s: pd.Series(s).apply(lambda modifier: '+' if modifier > 0 else '-' if modifier < 0 else '')
wearables_df[TRAIT_NAMES] = wearables_df['traitModifiers'].apply(pd.Series)
wearables_df[traitEffectColumns] = wearables_df['traitModifiers'].apply(getTraitEffect)
wearables_df.drop(['traitModifiers', 'EYS', 'EYC', 'EYS Effect', 'EYC Effect'], axis=1, inplace=True)

# slot positions
def get_slot_name_if_true(t):
    i, has_slot = t
    return SLOT_POSITION_NAMES[i] if has_slot else ''
is_not_empty = lambda x: x != ''
get_slot_names = lambda row: ', '.join(filter(is_not_empty, list(map(get_slot_name_if_true, enumerate(row)))))
wearables_df['slotNames'] = wearables_df['slotPositions'].apply(get_slot_names)
wearables_df.drop('slotPositions', axis=1, inplace=True)

# rarity
wearables_df['rarity'] = wearables_df['rarityScoreModifier'].apply(lambda x: RARITY_SCORE_MODIFIERS[x])


In [9]:
wearables_df.head(12)

,name,maxQuantity,rarityScoreModifier,NRG,AGG,SPK,BRN,NRG Effect,AGG Effect,SPK Effect,BRN Effect,slotNames,rarity
id,,,,,,,,,,,,,
1,Camo Hat,1000,1,0,1,0,0,,+,,,Head,common
2,Camo Pants,1000,1,0,1,0,0,,+,,,Body,common
3,MK2 Grenade,1000,1,0,1,0,0,,+,,,"Hand L, Hand R",common
4,Snow Camo Hat,500,2,0,1,1,0,,+,+,,Head,uncommon
5,Snow Camo Pants,500,2,0,1,1,0,,+,+,,Body,uncommon
6,M67 Grenade,500,2,0,2,0,0,,+,,,"Hand L, Hand R",uncommon
7,Marine Cap,300,5,0,2,0,1,,+,,+,Head,rare
8,Marine Jacket,300,5,0,2,0,1,,+,,+,Body,rare
9,Walkie Talkie,300,5,1,0,1,1,+,,+,+,"Hand L, Hand R",rare


In [10]:
wearables_df.describe()

,maxQuantity,rarityScoreModifier,NRG,AGG,SPK,BRN
count,246.000000,246.000000,246.000000,246.000000,246.000000,246.000000
mean,339.894309,11.186992,0.032520,0.020325,0.052846,0.121951
std,333.655653,13.730258,1.540925,1.547741,1.278253,1.443269
min,5.000000,1.000000,-6.000000,-6.000000,-5.000000,-5.000000
25%,100.000000,2.000000,0.000000,0.000000,0.000000,0.000000
50%,250.000000,5.000000,0.000000,0.000000,0.000000,0.000000
75%,500.000000,10.000000,1.000000,1.000000,0.000000,0.000000
max,1000.000000,50.000000,6.000000,6.000000,5.000000,6.000000


In [11]:
wearables_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 246 entries, 1 to 315
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   name                 246 non-null    object
 1   maxQuantity          246 non-null    int64 
 2   rarityScoreModifier  246 non-null    int64 
 3   NRG                  246 non-null    int64 
 4   AGG                  246 non-null    int64 
 5   SPK                  246 non-null    int64 
 6   BRN                  246 non-null    int64 
 7   NRG Effect           246 non-null    object
 8   AGG Effect           246 non-null    object
 9   SPK Effect           246 non-null    object
 10  BRN Effect           246 non-null    object
 11  slotNames            246 non-null    object
 12  rarity               246 non-null    object
dtypes: int64(6), object(7)
memory usage: 26.9+ KB


In [12]:
wearables_df.to_csv('db/wearable_types.csv')

purchases_query = get_core_matic_query(
    { 'erc1155Purchases': {
        'params': { 'where': { 'category': ITEM_TYPE_CATEGORY_WEARABLE } },
        'fields': ["id", "listingID", "category", "erc1155TypeId", "priceInWei", "quantity", "timeLastPurchased"] }}
)

purchases_result = purchases_query.execute(USE_CACHE)
purchases_df = get_subgraph_result_df(purchases_result)
purchases_df.head(10)